# Analyzing Computer Science versus Business Management Introductory Course Professors Reviews and Their Trends Over Time

William Ingold, Erik Kelemen, Ashish Manda

## Introduction

## Grabbing Introductory Course Professors From UMD.io

In [127]:
import requests
import pandas as pd
professors_url = "https://api.umd.io/v1/professors"

In [34]:
import csv
from os import path

cmsc_professor_names_filepath = './data/cmsc_professor_names.csv'
bmgt_professor_names_filepath = './data/bmgt_professor_names.csv'

have_cmsc_professors = path.exists(cmsc_professor_names_filepath)
have_bmgt_professors = path.exists(bmgt_professor_names_filepath)

def read_professor_name_data(professor_filepath):
    """Reads the professor names and their courses from a CSV file.
    
    Args:
        professor_filepath: String holding a filepath to the professor csv file.
        
    Returns:
        A dictionary of professor names to a set of courses they have taught.
    """
    
    with open(professor_filepath, mode='r') as csv_file:
        csv_reader = csv.DictReader(csv_file)
        line_count = 0

        professors = {}

        for row in csv_reader:
            if line_count != 0:
                professors[row['name']] = set([course for course in row['courses'].split(' ')])
            line_count += 1

        return professors

def save_professor_data(professors, filepath):
    """Saves the professor names and their courses to a CSV file.
    
    Args:
        professors: A dictionary of professor name keys and a set of courses for values.
    """
    
    columns = ['name', 'courses']
    try:
        with open(filepath, 'w') as csvfile:
            writer = csv.DictWriter(csvfile, fieldnames=columns)
            writer.writeheader()
            
            for name, courses in professors.items():
                writer.writerow({'name': name, 'courses': ' '.join(courses)})
                
    except IOError:
        print("Error in writing the CSV file")

In [35]:
def get_professors_for_courses(course_ids):
    """Gets all the professors for the given course_ids and returns a list of them.
    
    Args:
        course_ids: A list of course ids (e.g. ['CMSC216', CMSC250']).
        
    Returns:
        List of professors that teach the given courses.
    
    """
    
    professors = {}
    
    for course_id in course_ids:
        params = {'course_id': course_id}

        response = requests.get(professors_url, params)
        
        if response.status_code == 200:

            for item in response.json():
                name = item['name']

                if name in professors:
                    professors[name].add(course_id)
                else:
                    professors[name] = {course_id}

    return professors

### Computer Science Professors

In [36]:
cmsc_course_ids = ["CMSC131", "CMSC132", "CMSC216", "CMSC250"]

# TODO: Dr. Eastman has taught CMSC131, per RateMyProfessor, but wasn't given via UMD.IO

# Only query the UMD.io API if we don't have the data
if not have_cmsc_professors:
    cmsc_professors = get_professors_for_courses(cmsc_course_ids)
    save_professor_data(cmsc_professors, cmsc_professor_names_filepath)
    have_cmsc_professors = True
else: 
    cmsc_professors = read_professor_name_data(cmsc_professor_names_filepath)

    if not cmsc_professors:
        print("Error response from umd.io API")

if 'Iason Filippou' in cmsc_professors:
    cmsc_professors.pop('Iason Filippou') # A typo of Jason Filippou from the database
    
print(cmsc_professors)

{'Fawzi Emad': {'CMSC132', 'CMSC250', 'CMSC131'}, 'Ilchul Yoon': {'CMSC132', 'CMSC216', 'CMSC131'}, 'Nelson Padua-Perez': {'CMSC132', 'CMSC216', 'CMSC131'}, 'Pedram Sadeghian': {'CMSC132', 'CMSC131'}, 'Anwar Mamat': {'CMSC132'}, 'Laurence Herman': {'CMSC132', 'CMSC216'}, 'A Shankar': {'CMSC216'}, 'Aditya Acharya': {'CMSC250'}, 'Alexander Brassel': {'CMSC250'}, 'Clyde Kruskal': {'CMSC250'}, 'David Sekora': {'CMSC250'}, 'Donald Perlis': {'CMSC250'}, 'Jason Filippou': {'CMSC250'}, 'Mohammad Nayeem Teli': {'CMSC250'}, 'Roger Eastman': {'CMSC250'}}


### Business Management Professors

In [37]:
bmgt_course_ids = ["BMGT110", "BMGT220", "BMGT221", "BMGT230"]

# Only query the UMD.io API if we don't have the data
if not have_bmgt_professors:
    bmgt_professors = get_professors_for_courses(bmgt_course_ids)
    save_professor_data(bmgt_professors, bmgt_professor_names_filepath)
    have_bmgt_professors = True
else:
    bmgt_professors = read_professor_name_data(bmgt_professor_names_filepath)

    if not bmgt_professors:
        print("Error response from umd.io API")

print(bmgt_professors)

{'Jeff Miller': {'BMGT110'}, 'Cody Hyman': {'BMGT220'}, 'Laurel Mazur': {'BMGT221', 'BMGT220'}, 'Progyan Basu': {'BMGT220'}, 'Viktoriya Zotova': {'BMGT220'}, 'Gary Bulmash': {'BMGT221'}, 'Gerald Ward': {'BMGT221'}, 'Ai Ren': {'BMGT230'}, 'Daehoon Noh': {'BMGT230'}, 'Erich Studer-Ellis': {'BMGT230'}, 'Huan Cao': {'BMGT230'}, 'Radu Lazar': {'BMGT230'}, 'Shubham Akshat': {'BMGT230'}, 'Ziwei Cao': {'BMGT230'}}


## Grabbing Reviews

### Setup Databases to Hold Review Data

In [85]:
import sqlite3
from sqlite3 import Error

db_filepath = './data/db/'
bmgt_rmp_db_filepath = db_filepath + 'bmgt_rmp.db'
cmsc_rmp_db_filepath = db_filepath + 'cmsc_rmp.db'

cmsc_rmp_db_exists = path.exists(cmsc_rmp_db_filepath)
bmgt_rmp_db_exists = path.exists(bmgt_rmp_db_filepath)

def create_connection(db_file):
    """Create a connection to the provided database file.
    
    Args:
        db_file: A string holding the filepath to a database.
    """
    
    conn = None
    print(db_file)
    try:
        conn = sqlite3.connect(db_file)
        return conn
    except Error as e:
        print(e)

    return conn

def execute_create_command(conn, sql_command, params=()):
    """Executes the provided sql_command on the provided database.
    
    Args:
        conn: The connection object to the database.
        sql_command: A string containing the SQL command.
        params: A tuple of potential parameters.
    """
    
    try:
        c = conn.cursor()
        c.execute(sql_command, params)
        
    except Error as e:
        print(e)
        
def execute_insert_command(conn, sql_command, params=()):
    """Executes the provided sql_command on the provided database.
    
    Args:
        conn: The connection object to the database.
        sql_command: A string containing the SQL command.
        params: A tuple of potential parameters.
    """
    
    try:
        c = conn.cursor()
        c.execute(sql_command, params)
        conn.commit()
        
        return c.lastrowid
    except Error as e:
        print(e)
        
def execute_query_command(conn, sql_command, params=()):
    """Executes the provided sql_command on the provided database.
    
    Args:
        conn: The connection object to the database.
        sql_command: A string containing the SQL command.
        params: A tuple of potential parameters.
    """
    
    try:
        c = conn.cursor()
        c.execute(sql_command, params)
        
        return c.fetchall()
    
    except Error as e:
        print(e)

In [105]:
def create_rmp_tables(rmp_conn):
    """Create the stats and review tables for RateMyProfessors data.
    
    Args:
        rmp_conn: Connection object to a RateMyProfessors database.
    """
    
    stats_table = """ CREATE TABLE IF NOT EXISTS professor_stats (
                        id INTEGER PRIMARY KEY,
                        first_name TEXT NOT NULL,
                        last_name TEXT NOT NULL,
                        full_name TEXT NOT NULL UNIQUE,
                        rating REAL,
                        take_again REAL,
                        difficulty REAL,
                        rating_count INTEGER NOT NULL,
                        gives_good_feedback INTEGER,
                        respected INTEGER,
                        lots_of_homework INTEGER,
                        accessible_outside_class INTEGER,
                        get_ready_to_read INTEGER,
                        participation_matters INTEGER,
                        skip_class_wont_pass INTEGER,
                        inspirational INTEGER,
                        graded_by_few_things INTEGER,
                        test_heavy INTEGER,
                        group_projects INTEGER,
                        clear_grading_criteria INTEGER,
                        hilarious INTEGER,
                        beware_of_pop_quizes INTEGER,
                        amazing_lectures INTEGER,
                        lecture_heavy INTEGER,
                        caring INTEGER,
                        extra_credit INTEGER,
                        so_many_papers INTEGER,
                        tough_grader INTEGER
                    ) """
    
    # TODO: Review id format? <professor last name>-<#> ?
    review_table = """ CREATE TABLE IF NOT EXISTS reviews (
                        id INTEGER PRIMARY KEY,
                        review_id TEXT NOT NULL UNIQUE,
                        course TEXT NOT NULL,
                        date INTEGER NOT NULL,
                        body TEXT NOT NULL,
                        thumb_up INTEGER,
                        thumb_down INTEGER,
                        quality REAL NOT NULL,
                        difficulty REAL NOT NULL,
                        take_again INTEGER NOT NULL,
                        for_credit INTEGER NOT NULL,
                        textbook_use INTEGER NOT NULL,
                        attendance INTEGER,
                        grade_received TEXT,
                        online_class INTEGER,
                        gives_good_feedback INTEGER,
                        respected INTEGER,
                        lots_of_homework INTEGER,
                        accessible_outside_class INTEGER,
                        get_ready_to_read INTEGER,
                        participation_matters INTEGER,
                        skip_class_wont_pass INTEGER,
                        inspirational INTEGER,
                        graded_by_few_things INTEGER,
                        test_heavy INTEGER,
                        group_projects INTEGER,
                        clear_grading_criteria INTEGER,
                        hilarious INTEGER,
                        beware_of_pop_quizes INTEGER,
                        amazing_lectures INTEGER,
                        lecture_heavy INTEGER,
                        caring INTEGER,
                        extra_credit INTEGER,
                        so_many_papers INTEGER,
                        tough_grader INTEGER
                   ) """
    
    execute_create_command(rmp_conn, stats_table)
    execute_create_command(rmp_conn, review_table)

cmsc_rmp_db = create_connection(cmsc_rmp_db_filepath)
bmgt_rmp_db = create_connection(bmgt_rmp_db_filepath)

create_rmp_tables(cmsc_rmp_db)
create_rmp_tables(bmgt_rmp_db)

./data/db/cmsc_rmp.db
./data/db/bmgt_rmp.db


In [87]:
def is_rmp_scraped(rmp_conn, professor_name):
    """Returns if the professor's RateMyProfessors page has been scraped already.
    
    Args:
        rmp_conn: Connection object to the appropriate rmp database.
        professor_name: String holding the professor's name.
    """
    
    first_name, last_name = professor_name.split()
    
    sql_command = """SELECT
                        first_name,
                        last_name
                    FROM
                        professor_stats ps
                    WHERE
                        first_name LIKE ? AND last_name LIKE ?"""
    
    params=('%'+first_name+'%', '%'+last_name+'%')
    
    result = execute_query_command(rmp_conn, sql_command, params)
    
    return result is not None

In [112]:
def insert_rmp_professor_stats(rmp_conn, professor_name, stats, tag_list):
    first_name, last_name = professor_name.split(' ')
    name_count = 3
    column_count = name_count + len(stats) + len(tag_list)
    
    # Question mark for each value to be filled, don't want a trailing comma
    question_marks = "?," * (column_count - 1)
    question_marks = question_marks + "?"
    
    tags = ','.join(tag_list.keys())
    
    insert_stats_sql = """ INSERT INTO professor_stats (
                            first_name,
                            last_name,
                            full_name,
                            rating,
                            take_again,
                            difficulty,
                            rating_count,
                            {tags}
                    )
                    VALUES({question_marks})
                    """.format(question_marks=question_marks, tags=tags)
    
    # TODO: Need to surround with % ?
    values = tuple([first_name, last_name, professor_name] + list(stats.values()) + list(tag_list.values()))
    
    return execute_insert_command(rmp_conn, insert_stats_sql, values)

## From RateMyProfessor

In [121]:
from bs4 import BeautifulSoup

ratemyprofessor_url = "https://www.ratemyprofessors.com/search.jsp"

params = {'queryoption':'HEADER', 'schoolID':'1270', 'queryBy':'teacherName', 'schoolName':'University+of+Maryland'}

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:82.0) Gecko/20100101 Firefox/82.0",
    "Access-Control-Allow-Origin": "*",
    "Access-Control-Allow-Headers": "Content-Type",
    "Access-Control-Allow-Methods": "GET"
}

tag_list = ['gives_good_feedback', 'respected', 'lots_of_homework', 'accessible_outside_class',
           'get_ready_to_read', 'participation_matters', 'inspirational',
           'graded_by_few_things', 'test_heavy', 'group_projects', 'clear_grading_criteria', 
           'hilarious', 'beware_of_pop_quizes', 'amazing_lectures', 'lecture_heavy', 'caring',
           'extra_credit', 'so_many_papers', 'tough_grader', 'skip_class_wont_pass']

text_tag_list = [' '.join(x.split('_')) for x in tag_list]
text_tag_list.remove('skip class wont pass')
text_tag_list.append("skip class? you won't pass.")

# both tag_list and text_tag_list in same order, and correspond to one another
text_tag_dict = {text_tag_list[i]: tag_list[i] for i in range(len(text_tag_list))}

review_meta_list = ['would_take_again', 'grade', 'textbook', 'online_class', 'for_credit', 'attendance']
review_text_meta_list = [' '.join(x.split('_')) for x in review_meta_list]
review_meta_dict = {review_text_meta_list[i]: review_meta_list[i] for i in range(len(review_meta_list))}
        
review_header_list = ['review_id', 'course', 'date', 'quality', 'difficulty', 'body',
                      'thumb_up', 'thumb_down'] + review_meta_list + tag_list 

{'would take again': 'would_take_again', 'grade': 'grade', 'textbook': 'textbook', 'online class': 'online_class', 'for credit': 'for_credit', 'attendance': 'attendance'}
['review_id', 'course', 'date', 'quality', 'difficulty', 'body', 'thumb_up', 'thumb_down', 'would_take_again', 'grade', 'textbook', 'online_class', 'for_credit', 'attendance', 'gives_good_feedback', 'respected', 'lots_of_homework', 'accessible_outside_class', 'get_ready_to_read', 'participation_matters', 'inspirational', 'graded_by_few_things', 'test_heavy', 'group_projects', 'clear_grading_criteria', 'hilarious', 'beware_of_pop_quizes', 'amazing_lectures', 'lecture_heavy', 'caring', 'extra_credit', 'so_many_papers', 'tough_grader', 'skip_class_wont_pass']


In [44]:
def find_rmp_professor_url(html_doc):
    """Finds the professor's URL on the search page and returns it.
    
    Args:
        html_doc: A string containing an HTML document.
        
    Returns:
        The full URL for the professor's page (if found).
    
    """
    
    soup = BeautifulSoup(html_doc, 'html.parser')
    partial_url = soup.find('li', class_='listing PROFESSOR').find('a', href=True)
    
    if partial_url:
        main_url = "https://www.ratemyprofessors.com"
        return main_url + partial_url['href']

In [45]:
def query_rmp_for_professor_url(professor_name, headers, params):
    """Queries RateMyProfessor for the professor, given the parameters and headers.
    
    Args:
        professor_name: The <first name> <last name> of the professor.
        headers: Dictionary of headers for the get request.
        params: Dictionary of parameters for the get request.
        
    Returns:
        The full URL for the professor's page after searching for it (if found).
        
    """
    
    params['query'] = professor_name
    
    response = requests.get(ratemyprofessor_url, headers=headers, params=params)
    
    if response.status_code == 200:
        return find_rmp_professor_url(response.text)

In [75]:
def get_rmp_prof_stats(page_text):
    """Parses the professor's stats from their page and returns them. Namely their overall rating, 
    how many would take again, overall difficulty and how many ratings they have on RateMyProfessor.
    
    Args:
        page_text: An HTML document of the professor's page.
        
    Returns:
        A dictionary containing their rating, take again percentage, difficulty rating, and rating count.
    """
    
    soup = BeautifulSoup(page_text, 'html.parser')
    
    rating_score = float(soup.select('div[class*="RatingValue__Numerator"]')[0].text)
    
    feedback = soup.select('div[class*="TeacherFeedback__StyledTeacherFeedback"]')[0].select('div[class*="FeedbackItem__FeedbackNumber"]')
    
    take_again = float(feedback[0].text[:-1]) / 100
    difficulty = float(feedback[1].text)
    
    rating_count = soup.select('div[class*="RatingValue__NumRatings"]')[0].select('a')[0].text
    rating_count = ''.join([x for x in rating_count if x.isdigit()])
    rating_count = int(rating_count)
    
    return {'rating': rating_score, 'take_again': take_again, 'difficulty': difficulty, 'rating_count': rating_count}

In [139]:
def tags_to_dict(provided_tags):
    """Turns the list of text tags (e.g. skip class? you won't pass) into a dictionary
    of approriately named tags that work for database columns and if they were present.
    
    Args:
        provided_tags: A list of space separated tags scraped from the RMP page.
        
    Returns:
        A dictionary of {tag: 1 or 0} on whether a tag was used to describe the professor.
    """
    
    tag_dict = {val: 0 for val in text_tag_dict.values()}
    
    for tag in provided_tags:
        if tag.lower() in text_tag_dict.keys():
            tag_dict[text_tag_dict[tag.lower()]] = 1
            
    return tag_dict

def meta_to_dict(provided_meta):
    """Turns the dictionary of meta tags (e.g. Would Take Again: No) into a dictionary
    of appropriately named tags that work for database columns and values if they were present.
    
    Args:
        provided_meta: A dictionary of meta information from a review.
        
    Returns:
        A dictionary of {meta: 1 or 0} on whether a meta was used on the review.
    """
    
    meta_dict = {val: 0 for val in review_meta_list}
    
    for meta, response in provided_meta.items():
        if meta.lower() in review_meta_dict.keys():
            
            if response.lower() == "yes" or response.lower() == "mandatory":
                value = 1
                
            if meta.lower() == "grade":
                value = response
            
            meta_dict[review_meta_dict[meta.lower()]] = value
            
    return meta_dict

In [48]:
def get_rmp_prof_top_tags(page_text):
    """Parses and returns the professor's top tags.
    
    Args:
        page_text: An HTML document of the professor's page.
        
    Returns:
        A list of tags describing the professor.
    """
    
    soup = BeautifulSoup(page_text, 'html.parser')
    
    tags = []
    unparsed_tags = soup.select('div[class*="TeacherTags__TagsContainer"]')[0].select('span')
    
    for tag in unparsed_tags:
        tags.append(tag.text)
        
    return tags_to_dict(tags)

#### Need Selenium because Javascript hides more reviews

In [49]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import time

driver = webdriver.Firefox(executable_path='./bin/geckodriver.exe')

In [119]:
def load_all_rmp_reviews(page_url):
    """Loads all the reviews for a given porfessor and returns the text of all of them.
    
    Args:
        page_url: The URL for the professor's page.
        
    Returns:
        A string containing the HTML for all the reviews.
    """
    
    driver.get(page_url)
    
    # RateMyProfessors has a cookies pop up that overlays the website, it needs to be closed first
    time.sleep(0.5)
    close_cookies = driver.find_elements(By.XPATH, '//button[text()="Close"]')
    
    if close_cookies:
        close_cookies[0].click()
        
    load_more = driver.find_elements(By.XPATH, '//button[text()="Load More Ratings"]')
    
    # RateMyProfessors paginates the reviews via Javascript, so we must continually load more while the button is present
    while load_more:
        load_more[0].click()
        time.sleep(1)
        load_more = driver.find_elements(By.XPATH, '//button[text()="Load More Ratings"]')
        
    return driver.find_element_by_id('ratingsList').get_attribute('outerHTML')

In [158]:
def parse_rating_header(soup):
    """Parses and returns the rating header for a single review.
    
    Args:
        soup: An initialized BeautifulSoup object for the professor's page.
        
    Returns:
        A dictionary containing the course and date for the review.
    """
    
    rating_header = soup.select('div[class*="Rating__RatingInfo"]')
    
    if len(rating_header) != 0:
        course = rating_header[0].select('div[class*="RatingHeader__StyledClass"]')[0].text.strip()
        date = rating_header[0].select('div[class*="TimeStamp__StyledTimeStamp"]')[0].text.strip()
    else:
        print(soup)
    
    return {'course': course, 'date': date}

def parse_meta_data(soup):
    """Parses and returns the meta data for a single review.
    
    Args:
        soup: An initialized BeautifulSoup object for the professor's page.
        
    Returns:
        A dictionary containing the meta data (e.g. Would Take Again) for the review.
    """
    
    course_meta = soup.select('div[class*="CourseMeta__StyledCourseMeta"]')[0]
    review_meta_data = {}

    for meta_div in course_meta.select('div'):
        meta_data = meta_div.text.split(':')
        meta_name = meta_data[0].strip()
        meta_value = meta_data[1].strip()

        review_meta_data[meta_name] = meta_value

    return meta_to_dict(review_meta_data)

def parse_rating_data(soup):
    """Parses and returns the rating data for a single review.
    
    Args:
        soup: An initialized BeautifulSoup object for the professor's page.
        
    Returns:
        A dictionary containing the rating data for the quality and difficulty for the review.
    """
    
    rating_values_text = soup.select('div[class*="RatingValues__StyledRatingValues"]')[0].select('div[class*="RatingValues__RatingValue"]')
    quality = rating_values_text[0].text
    difficulty = rating_values_text[1].text

    rating_data = {'quality': quality, 'difficulty': difficulty}
    
    return rating_data

def parse_review_tags(soup):
    """Parses and returns the tags for a single review.
    
    Args:
        soup: An initialized BeautifulSoup object for the professor's page.
        
    Returns:
        A list containing the tags for the review.
    """
    
    tag_container = soup.select('div[class*="RatingTags__StyledTags"]')
    
    if tag_container: # Since not all reviews add tags
        unparsed_tags = tag_container[0].select('span')

        tags = []
        for tag in unparsed_tags:
            tags.append(tag.text)

        return tags_to_dict(tags)
    
def parse_thumb_scoring(soup):
    """Parses and returns the thumb scoring data for a single review.
    
    Args:
        soup: An initialized BeautifulSoup object for the professor's page.
        
    Returns:
        A dictionary containing the thumb scoring data for the review.
    """
    
    thumb_container = soup.select('div[class*="RatingFooter__StyledRatingFooter"]')[0].select('div[class*="RatingFooter__HelpTotal"]')

    thumb_up = thumb_container[0].text
    thumb_down = thumb_container[1].text
    thumb_data = {'thumb-up': thumb_up, 'thumb-down': thumb_down}

    return thumb_data

def parse_review_text(soup):
    """Parses and returns the review body text for a single review.
    
    Args:
        soup: An initialized BeautifulSoup object for the professor's page.
        
    Returns:
        A string containing the review text for the review.
    """
    
    review_text = soup.select('div[class*="Comments__StyledComments"]')[0].text
    
    return {'body': review_text}
    
def parse_single_rmp_review(review_item, courses):
    """Parses and returns all data for a single review.
    Namely it returns: Meta data, rating data, tags, thumb_scoring, and review text.
    
    Args:
        review_item: A single review list item containing all the appropraite HTML.
        
    Returns:
        A dictionary containing the meta data, rating data, tags, thumb_scoring, and review text
        for a single review.
    """
    
    soup = BeautifulSoup(review_item, 'html.parser')
    
    course_and_date = parse_rating_header(soup)
    
    # TODO: Loses course reviews like 'CMSC131CMSC132' where students combined multiple courses they took
    if course_and_date['course'] in courses:
        
        # Meta data
        meta_data = parse_meta_data(soup)
        
        # Rating data
        rating_data = parse_rating_data(soup)
        
        # Tags 
        tags = parse_review_tags(soup)
        
        # Thumb Scoring
        thumb_scoring = parse_thumb_scoring(soup)
        
        # Review body
        review_text = parse_review_text(soup)
        
        return {'meta_data': meta_data, 'rating_data': rating_data, 'tags': tags, 'thumb_scoring': thumb_scoring, 'review_text': review_text}

In [162]:
# TODO: Parse/scrape an entire professor's review listing and insert into the database(s)
def get_rmp_prof_reviews(rmp_prof_url, prof_name, prof_courses):
    reviews_html = load_all_rmp_reviews(rmp_prof_url)
    soup = BeautifulSoup(reviews_html, 'html.parser')
    
    review_id_head = prof_name.split(' ')[1] + '-'
    counter = 1
    
    review_df = pd.DataFrame(columns=review_header_list)
    
    for review in soup.find_all('li'):
        
        if len(review.select('div[class*="Rating__StyledRating"]')) != 0: # Avoid advertisement list items
            data = parse_single_rmp_review(str(review), prof_courses)

            if data: # Since the review could be of an undesired course
                flattened_data = {}

                for data_type, data_dict in data.items():
                    if not data_dict:
                        print(data_type)
                    for key, val in data_dict.items():
                        flattened_data[key] = val

                print(flattened_data)
                print()
    
    return None

In [117]:
# TODO: Parse/scrape an entire professor's RateMyProfessors page
def parse_rmp_page(rmp_prof_url, headers):
    rmp_prof_page = requests.get(rmp_prof_url, headers=headers)
    
    if rmp_prof_page.status_code == 200:
        soup = BeautifulSoup(rmp_prof_page.text, 'html.parser')
        
        stats_container = soup.select('div[class*="TeacherInfo__StyledTeacher"]')[0]
        prof_stats = get_rmp_prof_stats(stats_container.text)
        prof_tags = get_rmp_prof_top_tags(stats_container.text)
        
        all_reviews = get_rmp_prof_reviews(rmp_prof_url)
        
        
    return None

### Computer Science Professors

In [53]:
fawzi_url = query_rmp_for_professor_url('Fawzi Emad', headers, params)
print(fawzi_url)

response = requests.get(fawzi_url, headers=headers)

https://www.ratemyprofessors.com/ShowRatings.jsp?tid=313062


Split for testing purposes, don't want to query the page multiple times

In [93]:
# TODO: Should have a function that gets the appropriate sections of the response.text then sends to respective functions
if response.status_code == 200:
    fawzi_stats = get_rmp_prof_stats(response.text)
    fawzi_tags = get_rmp_prof_top_tags(response.text)
    print(fawzi_stats)
    print()
    print(fawzi_tags)
else:
    print('error')

{'rating': 4.4, 'take_again': 0.83, 'difficulty': 3.1, 'rating_count': 114}

{'gives_good_feedback': 0, 'respected': 1, 'lots_of_homework': 0, 'accessible_outside_class': 0, 'get_ready_to_read': 0, 'participation_matters': 0, 'inspirational': 0, 'graded_by_few_things': 0, 'test_heavy': 0, 'group_projects': 0, 'clear_grading_criteria': 0, 'hilarious': 1, 'beware_of_pop_quizes': 0, 'amazing_lectures': 1, 'lecture_heavy': 0, 'caring': 0, 'extra_credit': 0, 'so_many_papers': 0, 'tough_grader': 0, 'skip_class_wont_pass': 1}


In [163]:
eastman_url = query_rmp_for_professor_url('Roger Eastman', headers, params)

get_rmp_prof_reviews(eastman_url, 'Roger Eastman', ['CMSC131', 'CMSC250'])

{'would_take_again': 1, 'grade': 0, 'textbook': 1, 'online_class': 0, 'for_credit': 1, 'attendance': 1, 'quality': '5.0', 'difficulty': '5.0', 'gives_good_feedback': 0, 'respected': 0, 'lots_of_homework': 0, 'accessible_outside_class': 1, 'get_ready_to_read': 1, 'participation_matters': 0, 'inspirational': 0, 'graded_by_few_things': 0, 'test_heavy': 0, 'group_projects': 0, 'clear_grading_criteria': 0, 'hilarious': 0, 'beware_of_pop_quizes': 0, 'amazing_lectures': 0, 'lecture_heavy': 0, 'caring': 1, 'extra_credit': 0, 'so_many_papers': 0, 'tough_grader': 0, 'skip_class_wont_pass': 0, 'thumb-up': ' 0', 'thumb-down': ' 0', 'body': 'Fantastic professor. Was not the best for first time coders. CMSC131 was tough. Loved Eastman, met with him constantly and he was always willing to help.'}

{'would_take_again': 1, 'grade': 'B', 'textbook': 'B', 'online_class': 0, 'for_credit': 1, 'attendance': 1, 'quality': '4.0', 'difficulty': '4.0', 'gives_good_feedback': 1, 'respected': 0, 'lots_of_homework

AttributeError: 'NoneType' object has no attribute 'items'

In [58]:
single_review = """
<div class="Rating__RatingBody-sc-1rhvpxz-0 dGrvXb"><div class="RatingHeader__StyledHeader-sc-1dlkqw1-0 uBHCj"><div class="RatingHeader__ClassInfoWrapper-sc-1dlkqw1-1 jxOApy"><div class="RatingHeader__StyledClass-sc-1dlkqw1-2 gxDIt"><img src="/static/media/computer-icon.17c26169.svg" alt="Computer Icon" data-for="GLOBAL_TOOLTIP" data-tip="Online Class" data-tooltip="true" class="OnlineCourseLogo__StyledLogo-qyf3kt-0 gemNec" currentitem="false"> CMSC131</div><div class="EmotionLabel__StyledEmotionLabel-sc-1u525uj-0 cJfJJi"><span role="img" aria-label="Sunglasses">😎</span>awesome</div></div><div class="TimeStamp__StyledTimeStamp-sc-9q2r30-0 bXQmMr RatingHeader__RatingTimeStamp-sc-1dlkqw1-3 BlaCV">Dec 3rd, 2020</div></div><div class="RatingValues__StyledRatingValues-sc-6dc747-0 bJSTHc"><div class="RatingValues__RatingContainer-sc-6dc747-1 DObVa"><div class="RatingValues__RatingLabel-sc-6dc747-2 gLxTSP">Quality</div><div class="RatingValues__RatingValue-sc-6dc747-3 kLWEWI">5.0</div></div><div class="RatingValues__RatingContainer-sc-6dc747-1 DObVa"><div class="RatingValues__RatingLabel-sc-6dc747-2 gLxTSP">Difficulty</div><div class="RatingValues__RatingValue-sc-6dc747-3 jILzuI">1.0</div></div></div><div class="Rating__RatingInfo-sc-1rhvpxz-3 kEVEoU"><div class="RatingHeader__StyledHeader-sc-1dlkqw1-0 fUEMJm"><div class="RatingHeader__ClassInfoWrapper-sc-1dlkqw1-1 jxOApy"><div class="RatingHeader__StyledClass-sc-1dlkqw1-2 gxDIt"><img src="/static/media/computer-icon.17c26169.svg" alt="Computer Icon" data-for="GLOBAL_TOOLTIP" data-tip="Online Class" data-tooltip="true" class="OnlineCourseLogo__StyledLogo-qyf3kt-0 gemNec" currentitem="false"> CMSC131</div><div class="EmotionLabel__StyledEmotionLabel-sc-1u525uj-0 cJfJJi"><span role="img" aria-label="Sunglasses">😎</span>awesome</div></div><div class="TimeStamp__StyledTimeStamp-sc-9q2r30-0 bXQmMr RatingHeader__RatingTimeStamp-sc-1dlkqw1-3 BlaCV">Dec 3rd, 2020</div></div><div class="CourseMeta__StyledCourseMeta-x344ms-0 fPJDHT"><div class="MetaItem__StyledMetaItem-y0ixml-0 LXClX">Would Take Again: <span>Yes</span></div><div class="MetaItem__StyledMetaItem-y0ixml-0 LXClX">Grade: <span>A</span></div><div class="MetaItem__StyledMetaItem-y0ixml-0 LXClX">Textbook: <span>No</span></div><div class="MetaItem__StyledMetaItem-y0ixml-0 LXClX">Online Class: <span>Yes</span></div></div><div class="Comments__StyledComments-dzzyvm-0 gRjWel">Fawzi is a legend</div><div class="RatingTags__StyledTags-sc-1boeqx2-0 eLpnFv"><span class="Tag-bs9vf4-0 hHOVKF">Hilarious</span><span class="Tag-bs9vf4-0 hHOVKF">Lecture heavy</span></div><div class="RatingFooter__StyledRatingFooter-ciwspm-0 dbULCX"><div class="RatingFooter__ButtonWrapper-ciwspm-1 cwcCIQ"><div class="RatingFooter__HelpTotal-ciwspm-2 kAVFzA"><img src="/static/media/thumbs-up-black.eddae738.svg" class="VoteThumb__StyledVoteThumb-p2gtch-0 jCbELu" data-tooltip="true" data-tip="Helpful" data-for="GLOBAL_TOOLTIP" alt="Thumbs up"> 0</div><div class="RatingFooter__HelpTotal-ciwspm-2 kAVFzA"><img src="/static/media/thumbs-down-black.bd601b36.svg" class="VoteThumb__StyledVoteThumb-p2gtch-0 jCbELu" data-tooltip="true" data-tip="Not helpful" data-for="GLOBAL_TOOLTIP" alt="Thumbs down"> 0</div></div><div class="RatingFooter__ButtonWrapper-ciwspm-1 cwcCIQ"><a href="https://www.ratemyprofessors.com//flagTeacherRating.jsp?rid=33939742" class="ReportFlag__StyledReportFlag-sc-1c42epr-0 hjlYuE" data-tooltip="true" data-tip="Report this rating" data-for="GLOBAL_TOOLTIP" data-testid="reportflag_test_id" aria-disabled="false" currentitem="false"><div class="ReportFlag__FlagWrapper-sc-1c42epr-1 kVglhF"><svg xmlns="http://www.w3.org/2000/svg" width="24" height="24" viewBox="0 0 24 24"><defs><path id="prefix__a" d="M3.93 10c1.417 0 2.383.276 4.371 1.072 1.762.704 2.546.928 3.629.928 1.189 0 2.094-.165 2.754-.428.095-.039.177-.075.246-.108v-9.86c-.82.253-1.814.396-3 .396-1.417 0-2.383-.276-4.371-1.072C5.797.224 5.013 0 3.93 0 2.741 0 1.836.165 1.176.428A4.094 4.094 0 00.93.536v9.86c.82-.253 1.814-.396 3-.396z"></path></defs><g fill="none" fill-rule="evenodd"><path fill="#151515" fill-rule="nonzero" d="M3 3a1 1 0 01.293-.707c.22-.22.614-.483 1.21-.721C5.407 1.21 6.564 1 8 1c1.417 0 2.383.276 4.371 1.072C14.133 2.776 14.917 3 16 3c1.189 0 2.094-.165 2.754-.428.341-.137.508-.249.539-.28C19.923 1.663 21 2.11 21 3v12a1 1 0 01-.293.707c-.22.22-.614.483-1.21.721-.903.362-2.06.572-3.497.572-1.417 0-2.383-.276-4.371-1.072C9.867 15.224 9.083 15 8 15c-1.189 0-2.094.165-2.754.428a4.09 4.09 0 00-.247.108L5 22a1 1 0 01-2 0V3zm5 0c-1.189 0-2.094.165-2.754.428A4.094 4.094 0 005 3.536v9.86C5.82 13.143 6.814 13 8 13c1.417 0 2.383.276 4.371 1.072 1.762.704 2.546.928 3.629.928 1.189 0 2.094-.165 2.754-.428.095-.039.177-.075.246-.108v-9.86c-.82.253-1.814.396-3 .396-1.417 0-2.383-.276-4.371-1.072C9.867 3.224 9.083 3 8 3z"></path><g transform="translate(4.07 3)"><mask id="prefix__b" fill="#fff"><use xlink:href="#prefix__a"></use></mask><use fill="none" fill-rule="nonzero" xlink:href="#prefix__a"></use><g fill="none" mask="url(#prefix__b)"><path d="M0 0H64V64H0z" transform="translate(-25 -27)"></path></g></g></g></svg></div></a></div></div></div></div>
"""

print(parse_single_rmp_review(single_review, ["CMSC131"]))

{'meta_data': {'Would Take Again': 'Yes', 'Grade': 'A', 'Textbook': 'No', 'Online Class': 'Yes'}, 'rating_data': {'quality': '5.0', 'difficulty': '1.0'}, 'tags': {'gives_good_feedback': 0, 'respected': 0, 'lots_of_homework': 0, 'accessible_outside_class': 0, 'get_ready_to_read': 0, 'particpation_matters': 0, 'inspirational': 0, 'graded_by_few_things': 0, 'test_heavy': 0, 'group_projects': 0, 'clear_grading_criteria': 0, 'hilarious': 1, 'beware_of_pop_quizes': 0, 'amazing_lectures': 0, 'lecture_heavy': 1, 'caring': 0, 'extra_credit': 0, 'so_many_papers': 0, 'tough_grader': 0, 'skip_class_wont_pass': 0}, 'thumb_scoring': {'thumb-up': ' 0', 'thumb-down': ' 0'}, 'review_text': 'Fawzi is a legend'}


### Business Management Professors

## From Planetterp

### Computer Science Professors

### Business Management Professors

(Debug) Need to close these for now

In [115]:
cmsc_rmp_db.close()
bmgt_rmp_db.close()